In [ ]:
from pycsp import *
from csp import *
from csp import CSP

In [ ]:
def parse_input_file(input_file):
    with open(input_file, 'r') as f:
        lines = [line.strip() for line in f.readlines()]

    # Find the puzzles in the file
    puzzles = []
    current_puzzle = []
    for line in lines:
        if not line.startswith('#'):
            current_puzzle.append(line)
        elif line.startswith('# Start of puzzle'):
            current_puzzle = []
        elif line.startswith('# End of puzzle'):
            if current_puzzle:
                #rows, cols = [int(x) for x in current_puzzle.pop(0).split()]
                puzzles.append(current_puzzle)
                current_puzzle = []
    if current_puzzle:
        puzzles.append(current_puzzle)

    csp_problems = []

    for puzzle in puzzles:
        # Parse the puzzle dimensions
        rows, cols = [int(x) for x in puzzle.pop(0).split()]

        # Create the CSP problem
        csp = {}

        # Initialize the domain for each variable
        for i in range(rows):
            for j in range(cols):
                if puzzle[i][j] == '_':
                    csp[(i, j)] = set(['_', 'b'])
                else:
                    csp[(i, j)] = set([puzzle[i][j]])
        # Add the constraints
        for i in range(rows):
            for j in range(cols):
                if puzzle[i][j] != '_':
                    continue

                if 'b' in csp[(i, j)]:
                    for y in range(cols):
                        if puzzle[i][y] == 'b' and abs(y-j) == 1:
                            csp[(i, j)].discard('b')
                
                # No two bulbs may be adjacent in the same column
                if 'b' in csp[(i, j)]:
                    for x in range(rows):
                        if puzzle[x][j] == 'b' and abs(x-i) == 1:
                            csp[(i, j)].discard('b')

                # No more than one bulb may be placed in each row and each column
                # Wall constraints
                if puzzle[i][j].isdigit():
                    n_bulbs = int(puzzle[i][j])
                    # Count bulbs in the same row
                    count = 0
                    for y in range(cols):
                        if puzzle[i][y] == 'b':
                            count += 1
                    if count > n_bulbs:
                        csp[(i, j)].discard('b')
                    # Count bulbs in the same column
                    count = 0
                    for x in range(rows):
                        if puzzle[x][j] == 'b':
                            count += 1
                    if count > n_bulbs:
                        csp[(i, j)].discard('b')
       
        csp_problems.append((rows, cols, csp))
    
    return csp_problems


In [ ]:
def AC3(csp):
    """
    AC-3 algorithm for constraint propagation.
    """
    # Initialize queue with all arcs in CSP
    queue = []
    for constraint in csp.constraints:
        queue.append((constraint.var1, constraint.var2))
    
    # Perform arc consistency
    while queue:
        xi, xj = queue.pop(0)
        if revise(csp, xi, xj):
            if len(csp.domains[xi]) == 0:
                # Domain wiped out, return failure
                return False
            for xk in csp.neighbors[xi]:
                # Add arcs from Xi's neighbors to queue
                if xk != xj:
                    queue.append((xk, xi))
    return True


def revise(csp, xi, xj):
    """
    Helper function to revise domain of xi using constraints between xi and xj.
    """
    revised = False
    for xi_val in csp.domains[xi]:
        # Check if there is a consistent value in xj's domain for xi_val
        satisfies_constraint = False
        for xj_val in csp.domains[xj]:
            if all(constraint.is_satisfied(xi_val, xj_val) for constraint in csp.constraints_between(xi, xj)):
                satisfies_constraint = True
                break
        if not satisfies_constraint:
            # Remove inconsistent value from xi's domain
            csp.domains[xi].remove(xi_val)
            revised = True
    return revised


In [ ]:
def select_unassigned_variable(csp):
    """Returns the next unassigned variable to be considered for assignment."""
    for var in csp.variables:
        if var not in csp.assignment:
            return var
    return None


def order_domain_values(var, assignment, csp):
    """Returns a list of values in the domain of the given variable, ordered according to some heuristic."""
    domain = list(csp.domains[var])
    return domain


def is_consistent(var, value, assignment, csp):
    """Checks whether assigning a value to a variable would violate any constraints in the CSP."""
    # Check binary constraints with already assigned variables
    for neighbor in csp.neighbors[var]:
        if neighbor in assignment and not csp.constraints(var, value, neighbor, assignment[neighbor]):
            return False
    # Check unary constraints
    if value not in csp.domains[var]:
        return False
    return True


def assign(var, value, assignment):
    """Assigns a value to a variable in the current assignment."""
    assignment[var] = value


def unassign(var, assignment):
    """Removes a variable from the current assignment."""
    if var in assignment:
        del assignment[var]
        
def is_complete(assignment):
    for value in assignment.values():
        if value is None:
            return False
    return True




In [ ]:
def forward_check(csp):
    return rec_forward({}, csp)

def rec_forward(assignment, csp):
    if is_complete(assignment):
        return assignment

    var = select_unassigned_variable(csp)
    for value in order_domain_values(var, assignment, csp):
        if is_consistent(var, value, assignment, csp):
            assign(var, value, assignment)

            # Perform forward checking
            inferences = {}
            for neighbor in get_all_neighboring_vars(var, csp):
                if neighbor not in assignment:
                    inferences[neighbor] = set()
                    for neighbor_value in csp[neighbor]:
                        if not is_consistent(neighbor, neighbor_value, assignment, csp):
                            csp[neighbor].discard(neighbor_value)
                            inferences[neighbor].add(neighbor_value)

                    if not csp[neighbor]:
                        # If a variable has no legal values left, backtrack
                        unassign(var, assignment)
                        for (inferred_var, inferred_values) in inferences.items():
                            csp[inferred_var].update(inferred_values)
                        continue

            result = recursive_backtrack(assignment, csp)
            if result is not None:
                return result

            unassign(var, assignment)
            for (inferred_var, inferred_values) in inferences.items():
                csp[inferred_var].update(inferred_values)

    return None

In [ ]:
input_puzzle = "file_path"

csp_input=parse_input_file(input_puzzle)
#print(csp_input)
solution=forward_check(csp_input)
print(solution)